In [5]:
import sys
sys.path.append("/home/tromero_client/LazyMDP")
# training imports
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from wandb.integration.sb3 import WandbCallback
import wandb
import gymnasium as gym
# general imports
import argparse
import os
from lazywrapper import LazyWrapper, LastObservationWrapper

# define function
def make_custom_env(original_env, n_envs, seed, loaded_default_policy, penalty):
    def _init():
        env = LazyWrapper(LastObservationWrapper(gym.make(original_env)), default_policy=loaded_default_policy, penalty = penalty)
        return env
    return _init
# set parameters
default_model = PPO.load("/home/tromero_client/LazyMDP/baselines/suboptimal_pretrained_policies/ppo_BreakoutNoFrameskip-v4_0")
seed = 0
penalty = -1
original_env_name = "BreakoutNoFrameskip-v4"
n_envs = 4
# make environment
vec_env = make_vec_env(env_id=make_custom_env(original_env_name, n_envs, seed, default_model, penalty), n_envs=n_envs)
vec_obs = vec_env.reset()
# make original environment
original_env = make_vec_env(original_env_name, n_envs=4)
obs = original_env.reset()

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [6]:
original_env.step([0,1,0,1])

(array([[[[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
 
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
 
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
 
         ...,
 
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
 
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
 
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]]],
 
 
        [[[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0

In [7]:
vec_env.step([2,2,2,2])

(array([[[[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
 
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
 
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
 
         ...,
 
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
 
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
 
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]]],
 
 
        [[[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0

In [9]:
from stable_baselines3.common.evaluation import evaluate_policy

from stable_baselines3.common.atari_wrappers import AtariWrapper
# eval_env = Monitor(gym.make("Y"))
def make_internal_atari_env(original_env):
    def _init():
        env = AtariWrapper(gym.make(original_env))
        return env
    return _init
env_eval = make_vec_env(env_id=make_internal_atari_env("BreakoutNoFrameskip-v4"))
mean_reward, std_reward = evaluate_policy(default_model, env_eval, n_eval_episodes=10)
print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")



AssertionError: OpenCV is not installed, you can do `pip install opencv-python`

In [ ]:
vec_env = default_model.get_env()
obs = vec_env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render("human")

In [ ]:
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3 import A2C

# There already exists an environment generator
# that will make and wrap atari environments correctly.
def make_internal_atari_env(original_env):
    def _init():
        env = AtariWrapper(gym.make(original_env))
        return env
    return _init
# Here we are also multi-worker training (n_envs=4 => 4 environments)
vec_env = make_atari_env(env_id = make_internal_atari_env("BreakoutNoFrameskip-v4"), n_envs=4, seed=0)
# Frame-stacking with 4 frames
vec_env = VecFrameStack(vec_env, n_stack=4)

# model = A2C("CnnPolicy", vec_env, verbose=1)
# model.learn(total_timesteps=5_000)

obs = vec_env.reset()
# while True:
#     action, _states = model.predict(obs, deterministic=False)
#     obs, rewards, dones, info = vec_env.step(action)

5